In [4]:
import requests 
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
import re


In [8]:
url = "https://www.indeed.com/viewjob?jk=018ed0ca7a7ce1a2&tk=1deup3s155ov5804&from=serp&vjs=3"
driver = webdriver.Chrome(r'C:/Users/trang/Desktop/bootcamp/bootcampNYCDSA/Python/scrape_indeed/chromedriver_win32/chromedriver.exe')
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
driver.close()
posting = soup.find(name = "div", attrs = {"class":"jobsearch-JobComponent-description"}).text
posting = posting.lower()
posting
type(posting)

str

In [9]:
#get the page html
#parameter link to scrape
#output: beautiful soup object of html source
def get_soup(url):
    driver = webdriver.Chrome(r'C:/Users/trang/Desktop/bootcamp/bootcampNYCDSA/Python/scrape_indeed/chromedriver_win32/chromedriver.exe')
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.close()
    return soup

#getting link from the page source
#parameter: soup object
#output: the urls of job postings that need to be scraped
def get_joblinks(soup):
    urls = []
    
    #loop through all the posting links
    for link in soup.find_all('a', {'class': 'jobtitle'}):
                
        partial_url = link.get('href')
        
        #filter out sponsored posting
        if "www.indeed.com" in partial_url:
            url =  partial_url 
        elif '/pagead/' not in partial_url:
            url = "http://www.indeed.com" + partial_url
            urls.append(url)
    return urls

#get job links of a specified search with job title, number of page and location
#output:all of the urls of a certain number of page according to title and location
def get_search(title, num_page, location):
    
    #link generator
    base_url = 'https://www.indeed.com/jobs?q={}&l={}'.format(title, location)
    soup = get_soup(base_url)
    urls = get_joblinks(soup)
    #print(len(urls))
    
    #number of job postings in 1 search:
    posting_strcount = soup.find(name = "div", attrs = {"id" : "searchCount"}).text.strip()
    posting_strcount = posting_strcount[posting_strcount.find('of')+2:-4].strip()
   
    try:
        posting_count = int(posting_strcount)
    except ValueError:
        posting_count = int(posting_strcount.replace(',',''))
        pass
    #print(posting_count)
    
    
    #scrpae all of the pages
    for i in range(2, num_page +1):
        start_num = (i-1)*10
        base_url = "https://www.indeed.com/jobs?q={}&l={}&start={}".format(title, location,start_num)
        try:
            soup = get_soup(base_url)
            urls += get_joblinks(soup)
        except:
            continue
    return urls
    #print(urls)

    
###get text from each job posting
#parameter: url of a job posting
#output: return title, company name, and job description
def get_text(url):
    soup = get_soup(url)
    title = soup.find(name = 'h3').text
    company_name = soup.find(name = "h4", attrs ={ "class":"jobsearch-CompanyReview--heading"}).text
    posting = soup.find(name = "div", attrs = {"class":"jobsearch-JobComponent-description"}).text
    posting = posting.lower()
    
    return title, company_name, posting


##get data:get all of the job posting data and save in a json file
#parameter: title number of page and location
#output: get all data needed and save in json file
def get_data(j_title, num_page, location):
    
    #convert to to indeed format
    j_title = '+'.join(j_title.lower().split())
    
    jobs_dict = {}
    
    #get the urls for the parameters
    urls = get_search(j_title, num_page, location)
    
    #create the nested dictionary
    num_urls = len(urls)
    for i, url in enumerate(urls):
        try:
            title,company_name ,posting = get_text(url)
            jobs_dict[i] = {} #create nested dictionary
            jobs_dict[i]['title'] = title
            jobs_dict[i]['company name'] = company_name
            jobs_dict[i]['posting'] = posting
            jobs_dict[i]['url'] = url
            
        except:
            continue
            
        percent = (i+1) / num_urls
        print("Progress: {:2.0f}%".format(100*percent), end='\r')
    
    #save the dictionary as json file:
    file_name = j_title.replace('+','_') + '_'+location.replace(' ','_') + '.json'
    with open(file_name, 'w') as f:
        json.dump(jobs_dict, f)
        
    print('All {} postings have been scraped and saved!'.format(num_urls))
    
        
#get_data('data scientist', 10,'Seatle')   
            

cities = ['New York city', 'Boston','Austin', 'Chicago', 'Philadelphia', 'San Diego',
          'San Francisco', 'Seattle', 'Washington DC','Portland']


In [14]:
cities = [ 'Seattle', 'Washington DC','Portland']
for i in cities:
    get_data('data analyst', 10, i)
    #get_data('data scientist',10,i)
    #get_data('data engineer', 10, i)

All 100 postings have been scraped and saved!
All 100 postings have been scraped and saved!
All 99 postings have been scraped and saved!
